In [1]:
'''
   Trying MAB
'''

'\n   Trying MAB\n'

In [38]:
# lets get the data first
import numpy as np
import pandas as pd
import sqlalchemy

def get_connection():
    engine = sqlalchemy.create_engine('sqlite:////Users/nikhillondhe/projects/didactic-happiness/instance/tasks.db')
    return engine
    
def load_data():
    ''' Load data from sqlite db '''
    query = '''
         SELECT T.*, CAST(strftime('%H', R.rec_ts) as Integer) as rec_ts,
         CASE
             WHEN 
                 W.id IS NOT NULL 
                 AND cast((julianday(end_ts) - julianday(start_ts)) * 24 * 60 * 60 as Integer) > 60
             THEN 1
             ELSE 0
         END AS was_picked
         FROM task T
         INNER JOIN recommendation R
         ON T.id = R.task_id
         LEFT JOIN
         work_log W
         ON R.id = W.rec_id
    '''
    engine = get_connection()
    df = pd.read_sql(query, engine)
    engine.dispose()
    return df

In [39]:
from typing import Dict, List


class HashEncoder:
    def __init__(self, fields: List):
        self.fields = fields
        self.field_lookups = {f: {} for f in fields}

    def get_hash(self, input: Dict) -> int:
        encodes = []
        for f in self.fields:
            val = input.get(f, None)
            lookup = self.field_lookups[f]
            encoded = lookup.get(val, len(lookup))
            encodes.append(encoded)
            if val not in lookup:
                lookup[val] = encoded
        value = 0
        for e in encodes:
            value = value << 3
            value += e
        return value

def get_pending_tasks() -> pd.DataFrame:
    query = '''
       SELECT * FROM task
       WHERE task.status != 'done'
    '''
    engine = get_connection()
    df = pd.read_sql(query, engine)
    engine.dispose()
    return df

enc = HashEncoder(['complexity', 'type', 'priority', 'repeatable'])

def get_hash(row):
    return enc.get_hash(row.to_dict())
    
tasks = get_pending_tasks()
tasks['hash'] = tasks.apply(get_hash, axis=1)


In [40]:
tasks['hash'].value_counts()

hash
0       3
649     3
585     3
16      3
593     2
1169    2
144     2
9       2
192     2
65      2
584     1
1097    1
145     1
1       1
577     1
705     1
72      1
73      1
136     1
Name: count, dtype: int64

In [41]:
reward_col = 'was_picked'

df = load_data()

In [37]:
from collections import defaultdict

def compute_qvalues(df: pd.DataFrame, col: str, rewards: dict, reward_col: str) -> dict:
    values = df[col].unique()
    k = len(values)
    arm_to_idx = {v: k for k,v in enumerate(values)}
    Qvalues = np.zeros(k)
    counts = np.zeros(k)

    for idx, row in df.iterrows():
        arm_name = row[col]
        reward = rewards[arm_name] * row[reward_col]
        arm_idx = arm_to_idx[arm_name]
        counts[arm_idx] += 1
        Qvalues[arm_idx] += (reward - Qvalues[arm_idx]) / counts[arm_idx]

    
    return {k : Qvalues[v] for k, v in arm_to_idx.items()}

def compute_qvalues_v2(df: pd.DataFrame, reward_col: str, encoder: HashEncoder) -> dict:
    '''Compute qvalues using arm hashing'''
    Qvalues = defaultdict(float)
    counts = defaultdict(int)

    for _, row in df.iterrows():
        hash_code = encoder.get_hash(row.to_dict())
        reward = row[reward_col]
        counts[hash_code] += 1
        Qvalues[hash_code] += (reward - Qvalues[hash_code]) * 1.0 / counts[hash_code]

    return Qvalues

In [42]:
qvalues = compute_qvalues_v2(df, reward_col, enc)
qvalues

defaultdict(float,
            {585: 0.00597014925373134,
             577: 0.0,
             1: 0.08791208791208793,
             145: 0.02205882352941177,
             649: 0.04918032786885249,
             0: 0.15987460815047033,
             593: 0.08190476190476194,
             192: 0.04901960784313723,
             144: 0.08333333333333336,
             584: 0.010000000000000002,
             1097: 0.0,
             193: 0.38095238095238104,
             129: 0.050000000000000024,
             705: 0.0,
             16: 0.4166666666666668,
             17: 0.05882352941176471,
             73: 0.0,
             137: 0.11764705882352941,
             81: 0.07239819004524886,
             136: 0.14285714285714288,
             72: 0.0930232558139535,
             9: 0.1,
             65: 0.03448275862068967,
             1169: 0.05882352941176472})

In [43]:
# trying to do thomson sampling for alpha, beta comps

def compute_alpha_beta(df: pd.DataFrame, col: str, rewards: dict, reward_col: str) -> dict:
    #alpha += reward, beta += (exp reward - reward) but min(beta) = 1
    values = df[col].unique()
    distributions = {val : [1.0, 1.0] for val in values}
    exp_reward = sum(rewards.values()) * 1.0 / len(rewards)
    for _, row in df.iterrows():
        arm_name = row[col]
        distribution = distributions[arm_name]
        rewarded = row[reward_col]
        reward_value = rewards[arm_name]
        
        if rewarded:
            distribution[0] += reward_value
        else:
            distribution[1] += (exp_reward - reward_value)
            if distribution[1] < 1:
                distribution[1] = 1

    return distributions

def compute_alpha_beta_v2(df: pd.DataFrame, reward_col: str, encoder: HashEncoder) -> dict:
    '''Compute alpha and beta for thomspon sampling using hash encoded arms'''
    distributions = {}
    
    for _, row in df.iterrows():
        hash_code = encoder.get_hash(row.to_dict())
        reward = row[reward_col]
        distribution = distributions.get(hash_code, [0, 0])
        if reward:
            distribution[0] += 1
        else:
            distribution[1] += 1
        distributions[hash_code] = distribution

    return distributions

In [44]:
distributions = compute_alpha_beta_v2(df, reward_col, enc)
distributions

{585: [2, 333],
 577: [0, 81],
 1: [16, 166],
 145: [3, 133],
 649: [15, 290],
 0: [51, 268],
 593: [43, 482],
 192: [10, 194],
 144: [12, 132],
 584: [1, 99],
 1097: [0, 91],
 193: [8, 13],
 129: [7, 133],
 705: [0, 96],
 16: [25, 35],
 17: [4, 64],
 73: [0, 80],
 137: [2, 15],
 81: [32, 410],
 136: [7, 42],
 72: [4, 39],
 9: [9, 81],
 65: [1, 28],
 1169: [1, 16]}

In [59]:
data = df.copy(deep=True)
data['hash'] = data.apply(lambda row: enc.get_hash(row.to_dict()), axis=1)
data = data.groupby(['hash', reward_col]).size().unstack()
data.fillna(0, inplace=True)
for _, row in data.iterrows():
    print (row.loc[1])

51.0
16.0
9.0
25.0
4.0
1.0
4.0
0.0
32.0
7.0
7.0
2.0
12.0
3.0
10.0
8.0
0.0
1.0
2.0
43.0
15.0
0.0
0.0
1.0


In [55]:
def compute_ucb_values(df: pd.DataFrame, col: str, rewards: dict, reward_col: str) -> dict:
    '''Compute ucb values from historical data'''
    # avg reward + c * sqrt(ln(t) / num choiced)
    lnt = np.log(len(df))
    data = df.groupby([col, reward_col]).size().unstack()
    ucb = {}
    
    for k, v in rewards.items():
        n = data.loc[k].sum()
        reward = data.loc[k, 1] * v * 1.0
        avg_reward = reward / n
        ucb[k] = avg_reward + np.sqrt(lnt / n)
    return ucb

def compute_ucb_values_v2(df: pd.DataFrame, reward_col: str, encoder: HashEncoder) -> dict:
    lnt = np.log(len(df))
    data = df.copy(deep=True)
    data['hash'] = data.apply(lambda row: encoder.get_hash(row.to_dict()), axis=1)
    data = data.groupby(['hash', reward_col]).size().unstack()
    data.fillna(0, inplace=True)
    ucb = {}

    for _, row in data.iterrows():
        n = row.sum()
        reward = row.loc[1]
        avg_reward = reward * 1.0 / n
        ucb[row.name]= avg_reward + np.sqrt(lnt / n)
    return ucb

In [56]:
ucb = compute_ucb_values_v2(df, reward_col, enc)
ucb

{0: np.float64(0.32002188260858166),
 1: np.float64(0.29993324453472536),
 9: np.float64(0.4015043890544616),
 16: np.float64(0.785932620863173),
 17: np.float64(0.4056884949946473),
 65: np.float64(0.5656311791963122),
 72: np.float64(0.529218238237122),
 73: np.float64(0.31979369708687533),
 81: np.float64(0.20844982297515718),
 129: np.float64(0.29174131238222656),
 136: np.float64(0.5514745402641312),
 137: np.float64(0.8113769899892946),
 144: np.float64(0.32169348182074325),
 145: np.float64(0.26732939284910645),
 192: np.float64(0.2492821890948648),
 193: np.float64(1.0051257655835064),
 577: np.float64(0.31781353131654655),
 584: np.float64(0.29603217818489186),
 585: np.float64(0.16224622238073994),
 593: np.float64(0.206739438830987),
 649: np.float64(0.21296187665768912),
 705: np.float64(0.2919303694040929),
 1097: np.float64(0.299843195205764),
 1169: np.float64(0.7525534605775299)}

In [15]:
def compute_metrics(q_values):
    mean = np.mean(q_values)
    std_dev = np.std(q_values)
    cv = std_dev / mean if mean > 0 else 0
    entropy = -np.sum((q_values / np.sum(q_values)) * np.log(q_values / np.sum(q_values)))
    gini = np.sum(np.abs(np.subtract.outer(q_values, q_values))) / (2 * len(q_values) * np.sum(q_values))
    top_k_concentration = np.sum(sorted(q_values, reverse=True)[:2]) / np.sum(q_values)  # Top-2 concentration

    return {
        "mean": mean,
        "std_dev": std_dev,
        "cv": cv,
        "entropy": entropy,
        "gini": gini,
        "top_2_concentration": top_k_concentration,
    }

arm_types = {
    'complexity': [0.12806324, 0.08574673, 0],
    'priority': [0.08396125, 0.08361582, 0.57198444],
    'type': [0.36653895, 0.12583893, 0.72687225, 0.17578125]
}

# Evaluate each arm type
results = {arm_type: compute_metrics(q_values) for arm_type, q_values in arm_types.items()}

# Display results
for arm_type, metrics in results.items():
    print(f"{arm_type}: {metrics}")


complexity: {'mean': np.float64(0.07126998999999999), 'std_dev': np.float64(0.05327432377543676), 'cv': np.float64(0.7475000877008229), 'entropy': np.float64(nan), 'gini': np.float64(0.3993054829638362), 'top_2_concentration': np.float64(1.0)}
priority: {'mean': np.float64(0.24652050333333334), 'std_dev': np.float64(0.2301377998553537), 'cv': np.float64(0.9335442559281663), 'entropy': np.float64(0.6921823464455177), 'gini': np.float64(0.44023259133645287), 'top_2_concentration': np.float64(0.8869386536895356)}
type: {'mean': np.float64(0.34875784499999996), 'std_dev': np.float64(0.23606165298820578), 'cv': np.float64(0.6768640659200248), 'entropy': np.float64(1.1688721800791742), 'gini': np.float64(0.35731412364358434), 'top_2_concentration': np.float64(0.7837896807740627)}


/var/folders/63/vdsl0bvd413ff898d5q2jwb80000gq/T/ipykernel_45890/1635742412.py:5: RuntimeWarning: divide by zero encountered in log
  entropy = -np.sum((q_values / np.sum(q_values)) * np.log(q_values / np.sum(q_values)))
/var/folders/63/vdsl0bvd413ff898d5q2jwb80000gq/T/ipykernel_45890/1635742412.py:5: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum((q_values / np.sum(q_values)) * np.log(q_values / np.sum(q_values)))


In [ ]:
from typing import Dict, List
import random


class Recommender:
    def __init__(self, tasks: List):
        self.tasks = tasks

    def recommend(self, num_recs = 5) -> List:
        return random.sample(self.tasks, num_recs)

class MABRecommender(Recommender):
    def __init__(self, tasks: List):
        super.__init__(self, tasks)
        self.arms = {}
        for task in tasks:
            priority = task.get('priority')
            task_list = self.arms.get(priority, [])
            task_list.append(task)
            self.arms[priority] = task_list
        self.num_arms = len(self.arms)

    @abstractmethod
    def pull_arm(self) -> str:
        
    def recommend(self, num_recs = 5) -> List:
        for tasks in self.arms.values():
            random.shuffle(tasks)

        recs = {}
        recs_provided = 0
        arm_rec_count = {k : 0 for k in self.arms.keys()}
        max_recs = np.ceil(num_recs * 0.5)
        
        while recs_provided < num_recs:
            arm = self.pull_arm()

            if arm_rec_count[arm] == max_recs:
                
            task_list = self.arms[arm]
            for task in task_list:
                if task not in recs:
                    recs.add(task)
                    recs_provided += 1
                    arm_rec_count[arm] += 1
                    break
        return recs
                    
class EpsilonRecommender(MABRecommender):
    def __init__(self, tasks: List, qvalues: Dict): 
        super__init__(self, tasks)
        self.idx_to_arm = []
        self.qvalues = []
        for k, v in qvalues.items():
            self.qvalues.append(v)
            self.idx_to_arm.append(k)
        self.epsilon = 0.2

    def pull_arm(self):
        if random.random() < self.epsilon:
            idx = random.randint(0, self.num_arms-1)
        else:
            idx = np.argmax(self.qvalues)
        return self.idx_to_arm[idx]

class ThompsonSamplingRecommender(MABRecommender):
    def __init__(self, tasks: List, ts_values: Dict):
        super.__init__(self, tasks)
        self.ts_values = ts_values

    def pull_arm(self):
        for 

In [25]:
df

,id,name,complexity,type,due_date,priority,repeatable,status,rec_ts,was_picked
0,6,nlp scratch: seq 2 seq learn,medium,learning,None,medium,0,pending,16,0
1,7,attention in PyTorch,medium,learning,None,low,0,pending,16,0
2,15,read ml book,medium,learning,None,medium,0,done,16,0
3,8,tidy living room,simple,chores,None,low,0,done,16,0
4,16,RL: gaussian LL,medium,learning,None,medium,0,pending,16,0
...,...,...,...,...,...,...,...,...,...,...
3209,32,happiness: mab,medium,constructive,None,medium,0,in_progress,18,0
3210,47,boil eggs,simple,chores,None,low,1,pending,18,0
3211,39,face pack,simple,chores,None,low,1,pending,18,0
3212,4,vector search,hard,learning,None,medium,0,pending,18,0


In [32]:
ts = {'medium': [91.0, 631.6666666666657],
 'low': [79.0, 2289.0000000000095],
 'high': [750.0, 1]}

for k,v in ts.items():
    b = np.random.beta(v[0], v[1])
    print (f'{k} :: {b}')

medium :: 0.1089622191730933
low :: 0.03182924638764335
high :: 0.9977455966923572
